In [ ]:
from google.colab import drive 
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [5]:
%cd /content/gdrive/My Drive/ANN
!ls

/content/gdrive/My Drive/ANN
logs	       train_data.npy  X_train.pickle  Y_train.pickle
test_data.npy  X_test.pickle   Y_test.pickle


In [6]:
%tensorflow_version 

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.


In [7]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.callbacks import TensorBoard, EarlyStopping
import pickle
import time
import numpy as np
import keras.optimizers
from sklearn.metrics import classification_report

In [8]:
import tensorflow
print(tensorflow.__version__)

2.12.0


In [9]:
tensorboard = TensorBoard(log_dir='./logs', histogram_freq=0,
                          write_graph=True, write_images=False)
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5)


In [10]:
import pickle

pickle_in = open("X_test.pickle","rb")
X_test = pickle.load(pickle_in)

pickle_in = open("X_train.pickle","rb")
X_train = pickle.load(pickle_in)

pickle_in = open("Y_test.pickle","rb")
Y_test = pickle.load(pickle_in)

pickle_in = open("Y_train.pickle","rb")
Y_train = pickle.load(pickle_in)

import numpy as np

Y_train = np.array(Y_train)
Y_test = np.array(Y_test)

In [11]:
print(X_train.shape)
print(X_test.shape)

(2870, 150, 150, 3)
(394, 150, 150, 3)


In [12]:
dense_layers = [0,1,2]
layer_sizes = [32, 64, 128]

for dense_layer in dense_layers:
    for layer_size in layer_sizes:
      NAME = "{}-nodes-{}-dense-{}".format(layer_size, dense_layer, int(time.time()))
      print(NAME)

32-nodes-0-dense-1681317903
64-nodes-0-dense-1681317903
128-nodes-0-dense-1681317903
32-nodes-1-dense-1681317903
64-nodes-1-dense-1681317903
128-nodes-1-dense-1681317903
32-nodes-2-dense-1681317903
64-nodes-2-dense-1681317903
128-nodes-2-dense-1681317903


In [13]:
#%load_ext tensorboard
import os
import datetime
import tensorflow as tf
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback= tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

In [14]:
%reload_ext tensorboard

In [15]:
from tensorboard import notebook
notebook.list() # View open TensorBoard instances

notebook.display(port=6006, height=1000)

No known TensorBoard instances running.


<IPython.core.display.Javascript object>

In [16]:
dense_layers = [0,1, 2]
layer_sizes = [32, 64, 128]
for dense_layer in dense_layers:
    for layer_size in layer_sizes:
      #   
      NAME = "nodes-{}-dense-{}".format(layer_size, dense_layer, int(time.time()))
      print(NAME)
      model = Sequential()
      model.add(Flatten(input_shape=X_train.shape[1:]))
      for _ in range(dense_layer):
          model.add(Dense(layer_size))
          model.add(Activation('relu'))
          model.add(Dropout(0.33))

      model.add(Dense(4))
      model.add(Activation('softmax'))


      model.compile(loss='sparse_categorical_crossentropy',
        optimizer= "adam",
        metrics=['accuracy'],
        )
      
      #Fit the model

    #   validation_data: Data on which to evaluate the loss and any model metrics at the end of each epoch. The model will not be trained on this data. validation_data will override validation_split. validation_data could be:
    # •   tuple (x_val, y_val) of Numpy arrays or tensors
    # •   tuple (x_val, y_val, val_sample_weights) of Numpy arrays
    # •   dataset 
#   BATCH_SIZE = 100
# train_data = train_data.batch(BATCH_SIZE)
# validation_data = validation_data.batch(BATCH_SIZE)
# ...
# model.fit(train_data, epochs=NUM_EPOCHS, validation_data=validation_data,validation_steps=1)

      model.fit(X_train, Y_train,batch_size=32,epochs=20,validation_data=(X_test,Y_test),callbacks=[tensorboard_callback,es])
              
      #Score 
      scores = model.evaluate(X_test, Y_test, verbose=1) 
      print('Test loss:', scores[0])
      print('Test accuracy:', scores[1])
      
      #Save model
      model.save("{}-model-{}-accuracy.h5".format(NAME,scores[1]))
      
      #precision    recall  f1-score   support
      y_pred = model.predict(X_test, batch_size=64, verbose=1)
      y_pred_bool = np.argmax(y_pred, axis=1)
      print(classification_report(Y_test, y_pred_bool))
      print()
      print()  

nodes-32-dense-0
Epoch 1/20
90/90 [==============================] - 5s 9ms/step - loss: 737.0033 - accuracy: 0.4990 - val_loss: 816.3596 - val_accuracy: 0.4873
Epoch 2/20
90/90 [==============================] - 0s 5ms/step - loss: 247.8486 - accuracy: 0.6310 - val_loss: 1152.9259 - val_accuracy: 0.4162
Epoch 3/20
90/90 [==============================] - 0s 5ms/step - loss: 173.9633 - accuracy: 0.7000 - val_loss: 1317.6764 - val_accuracy: 0.4467
Epoch 4/20
90/90 [==============================] - 0s 5ms/step - loss: 137.7010 - accuracy: 0.7338 - val_loss: 1561.7820 - val_accuracy: 0.4594
Epoch 5/20
90/90 [==============================] - 0s 5ms/step - loss: 141.2111 - accuracy: 0.7669 - val_loss: 1593.3987 - val_accuracy: 0.5051
Epoch 6/20
90/90 [==============================] - 1s 7ms/step - loss: 116.0171 - accuracy: 0.7732 - val_loss: 1279.3904 - val_accuracy: 0.5635
Epoch 6: early stopping
13/13 [==============================] - 0s 4ms/step - loss: 1279.3904 - accuracy: 0.5635


/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch 1/20
90/90 [==============================] - 2s 17ms/step - loss: 235.6947 - accuracy: 0.2990 - val_loss: 6.4440 - val_accuracy: 0.1853
Epoch 2/20
90/90 [==============================] - 1s 16ms/step - loss: 1.4849 - accuracy: 0.2850 - val_loss: 12.6830 - val_accuracy: 0.1878
Epoch 3/20
90/90 [==============================] - 1s 16ms/step - loss: 1.4900 - accuracy: 0.2885 - val_loss: 2.0630 - val_accuracy: 0.1878
Epoch 4/20
90/90 [==============================] - 1s 15ms/step - loss: 1.3620 - accuracy: 0.2798 - val_loss: 2.0669 - val_accuracy: 0.1878
Epoch 5/20
90/90 [==============================] - 1s 15ms/step - loss: 1.3584 - accuracy: 0.2882 - val_loss: 2.0715 - val_accuracy: 0.1878
Epoch 6/20
90/90 [==============================] - 1s 15ms/step - loss: 1.3548 - accuracy: 0.2885 - val_loss: 2.0761 - val_accuracy: 0.1878
Epoch 7/20
90/90 [==============================] - 2s 18ms/step - loss: 1.3526 - accuracy: 0.2787 - val_loss: 2.0800 - val_accuracy: 0.1878
Epoch 8/20

/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch 1/20
90/90 [==============================] - 4s 36ms/step - loss: 354.0796 - accuracy: 0.3265 - val_loss: 5.0008 - val_accuracy: 0.3096
Epoch 2/20
90/90 [==============================] - 3s 31ms/step - loss: 1.4542 - accuracy: 0.2958 - val_loss: 2.3222 - val_accuracy: 0.2995
Epoch 3/20
90/90 [==============================] - 2s 26ms/step - loss: 1.3687 - accuracy: 0.2746 - val_loss: 2.3268 - val_accuracy: 0.2995
Epoch 4/20
90/90 [==============================] - 3s 29ms/step - loss: 1.3635 - accuracy: 0.2784 - val_loss: 2.3313 - val_accuracy: 0.2995
Epoch 5/20
90/90 [==============================] - 2s 27ms/step - loss: 1.3590 - accuracy: 0.2868 - val_loss: 2.3363 - val_accuracy: 0.2995
Epoch 6/20
90/90 [==============================] - 3s 30ms/step - loss: 1.3558 - accuracy: 0.2822 - val_loss: 2.3406 - val_accuracy: 0.2589
Epoch 7/20
90/90 [==============================] - 3s 35ms/step - loss: 1.3535 - accuracy: 0.2836 - val_loss: 2.3452 - val_accuracy: 0.2995
Epoch 7: ea

...

/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch 1/20
90/90 [==============================] - 3s 18ms/step - loss: 183.3692 - accuracy: 0.2937 - val_loss: 1.3942 - val_accuracy: 0.2538
Epoch 2/20
90/90 [==============================] - 1s 16ms/step - loss: 1.3623 - accuracy: 0.2906 - val_loss: 1.4062 - val_accuracy: 0.1878
Epoch 3/20
90/90 [==============================] - 1s 16ms/step - loss: 1.3546 - accuracy: 0.2885 - val_loss: 1.4166 - val_accuracy: 0.1878
Epoch 4/20
90/90 [==============================] - 1s 15ms/step - loss: 1.3518 - accuracy: 0.2857 - val_loss: 1.4249 - val_accuracy: 0.1878
Epoch 5/20
90/90 [==============================] - 2s 18ms/step - loss: 1.3495 - accuracy: 0.2923 - val_loss: 1.4305 - val_accuracy: 0.1878
Epoch 6/20
90/90 [==============================] - 2s 21ms/step - loss: 1.3489 - accuracy: 0.2906 - val_loss: 1.4358 - val_accuracy: 0.1878
Epoch 6: early stopping
13/13 [==============================] - 0s 4ms/step - loss: 1.4358 - accuracy: 0.1878
Test loss: 1.4358108043670654
Test accura

/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch 1/20
90/90 [==============================] - 4s 28ms/step - loss: 289.6289 - accuracy: 0.2986 - val_loss: 2.3441 - val_accuracy: 0.1802
Epoch 2/20
90/90 [==============================] - 2s 27ms/step - loss: 1.4318 - accuracy: 0.2892 - val_loss: 2.3538 - val_accuracy: 0.1802
Epoch 3/20
90/90 [==============================] - 3s 28ms/step - loss: 1.3542 - accuracy: 0.2878 - val_loss: 2.3627 - val_accuracy: 0.1802
Epoch 4/20
90/90 [==============================] - 3s 32ms/step - loss: 1.3515 - accuracy: 0.2864 - val_loss: 2.3682 - val_accuracy: 0.1802
Epoch 5/20
90/90 [==============================] - 3s 32ms/step - loss: 1.3487 - accuracy: 0.2854 - val_loss: 2.3725 - val_accuracy: 0.1802
Epoch 6/20
90/90 [==============================] - 2s 27ms/step - loss: 1.3489 - accuracy: 0.2948 - val_loss: 2.3772 - val_accuracy: 0.1802
Epoch 6: early stopping
13/13 [==============================] - 0s 3ms/step - loss: 2.3772 - accuracy: 0.1802
Test loss: 2.377188205718994
Test accurac

/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
